<a href="https://colab.research.google.com/github/TAUforPython/machinelearning/blob/main/Feed%20forward%20neural%20networks%20-%20example" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'uci-credit-approval-data-set:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F273093%2F565657%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240928%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240928T213928Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3bc9097e4e0e0132f2106365d9c43441b4d7a84868d941a28af67942f05a456aefaa141303d077edb28163a9e506133686995a735db9efa10f31a703455c693b38de26cff87806a1b57765ecee177b46070e71f14f9380534d2dec634b9f87e104d99aff92308f70138a845961a10dd11278c56b9edd2caeb79f8d013f06abcb68f9236307d9a0268e06446eb8a45b2b75330a1741efaeafdac2ca11c24feb732140ed6f592a7e09c983e975a7eb03c348706c7c8942f9565a5bcc765df2ce643d8ad7c252db6cda5cbd11b0710fd93dec2e04eb70304fa1e318be1f8cc3cc149b00f529b4d55e5d01604e88361fe5586fcd4beaf1e82ad01da2cc48f1e241d6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**Imports and workspace setting**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras import backend as K
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import f1_score
from tensorflow.keras.utils import plot_model

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%matplotlib inline

**Loading data**

In [ ]:
dataset = pd.read_csv('../input/uci-credit-approval-data-set/UCI_crx.csv')
dataset.shape

In [ ]:
dataset.head()

**Converting data types to tensor supported data types**

In [ ]:
dataset.dtypes

In [ ]:
for col in ['A1', 'A2','A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13', 'A14', 'A16']:
    dataset[col] = pd.Categorical(dataset[col])
    dataset[col] = dataset[col].cat.codes

In [ ]:
dataset.dtypes

In [ ]:
dataset.head()

**Normalizing input and output vectors**

Scale and normalize to make the magnitude of the features similar. Most popular methods for neumerical data are Min Max normalization and Standard normalization (z score). This will improve the model performance and convergence.

https://visualstudiomagazine.com/articles/2014/01/01/how-to-standardize-data-for-neural-networks.aspx
https://machinelearningmastery.com/how-to-improve-neural-network-stability-and-modeling-performance-with-data-scaling/

In [ ]:
# create scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dataset = pd.DataFrame(scaler.fit_transform(dataset))

dataset.describe()

In [ ]:
X=dataset.iloc[:,0:15].values   #0:15
Y=dataset.iloc[:,15:16].values

In [ ]:
np.random.seed(42) # Makes the random numbers predictable for easy comparison of models

**Create the model generator function**

Deciding neuron count in each layer and number of hidden layers is difficult and there is no straightforward answer to this.

https://www.heatonresearch.com/2017/06/01/hidden-layers.html

https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

In [ ]:
## Set these parameter before calling create_model function
depthOfNetwork = 3
neuronCountInEachLayer = [16, 9, 1]                                 # try different depth and width
activationFuncEachLayer = ['sigmoid', 'relu', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda


def create_model(verbose=False):
  model = tf.keras.models.Sequential()

  if verbose:
        print('Network configuration ',neuronCountInEachLayer)

  model.add(tf.keras.layers.Dense(neuronCountInEachLayer[0], input_dim=15, activation = activationFuncEachLayer[0], kernel_regularizer=regularizerFunc)) # First Layer

  for x in range(1, depthOfNetwork-1):
      model.add(tf.keras.layers.Dense(neuronCountInEachLayer[x], activation = activationFuncEachLayer[x],kernel_regularizer=regularizerFunc))         # Second layer onwards

  model.add(tf.keras.layers.Dense(neuronCountInEachLayer[depthOfNetwork-1], activation = activationFuncEachLayer[depthOfNetwork-1]))  # Output layer

  model.compile(loss = lossFunction , optimizer = 'adam' , metrics = ['accuracy'] )

  return model

**Visualizing a single model for ease of understanding**

In [ ]:
depthOfNetwork = 3
neuronCountInEachLayer = [17, 8, 1]                                 # try different depth and width
activationFuncEachLayer = ['sigmoid', 'relu', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

model=create_model()
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

**Define the evaluator function**

In [ ]:
def evaluateTheModel(verbose=False):
    n_split=5
    f1_scores = []

    for train_index,test_index in StratifiedKFold(n_split).split(X, Y):      # StratifiedKFold, KFold
        x_train,x_test=X[train_index],X[test_index]
        y_train,y_test=Y[train_index],Y[test_index]

        model=create_model(verbose)
        model.fit(x_train, y_train,epochs=100, verbose=0)
        evaluationMetrics = model.evaluate(x_test,y_test, verbose=0)

        if verbose:
            print('Model evaluation ',evaluationMetrics)   # This returns metric values for the evaluation

        y_pred = np.where(model.predict(x_test) > 0.5, 1, 0)
        f1 = f1_score(y_test, y_pred , average="macro")

        if verbose:
            print('F1 score is ', f1)

        f1_scores.append(f1)

    return np.mean(f1_scores)

**Verbose mode of the function to see F1 scores of each fold**

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [2, 1]                                 # try different depth and width
activationFuncEachLayer = ['sigmoid', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

evaluateTheModel(True)

**Experiment with chaning the first layer neuron count**

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [15, 1]                                 # try different depth and width
activationFuncEachLayer = ['sigmoid', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

for i in range (3, 20):
    neuronCountInEachLayer = [i, 1]
    print("'Node count : % 3d, Mean F1 score : % 10.5f" %(i, evaluateTheModel()))

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [15, 1]                                 # try different depth and width
activationFuncEachLayer = ['relu', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

for i in range (3, 20):
    neuronCountInEachLayer = [i, 1]
    print("'Node count : % 3d, Mean F1 score : % 10.5f" %(i, evaluateTheModel()))

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [15, 1]                                 # try different depth and width
activationFuncEachLayer = ['tanh', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

for i in range (3, 20):
    neuronCountInEachLayer = [i, 1]
    print("'Node count : % 3d, Mean F1 score : % 10.5f" %(i, evaluateTheModel()))

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [15, 1]                                 # try different depth and width
activationFuncEachLayer = ['tanh', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'mean_squared_error'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

for i in range (3, 20):
    neuronCountInEachLayer = [i, 1]
    print("'Node count : % 3d, Mean F1 score : % 10.5f" %(i, evaluateTheModel()))

**Experiment with chaning 2nd layer neuron count while keeping width to 15 neurons**

In [ ]:
depthOfNetwork = 3
neuronCountInEachLayer = [18, 9, 1]                                 # try different depth and width
activationFuncEachLayer = ['tanh', 'tanh', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

for i in range (15, 16):
    for j in range (3, 20):
        neuronCountInEachLayer = [i, j, 1]
        print("'Neurons [% 3d, % 3d], Mean F1 score : % 10.5f" %(i, j, evaluateTheModel()))

**Trying 3 layer setup**

In [ ]:
depthOfNetwork = 4
neuronCountInEachLayer = [15, 8, 5, 1]                                 # try different depth and width
activationFuncEachLayer = ['tanh', 'tanh', 'tanh','sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

print("'Neurons [% 3d, % 3d, % 3d], Mean F1 score : % 10.5f" %(3, 4, 3, evaluateTheModel()))

**Adding regularization to minimize overfitting**

https://machinelearningmastery.com/how-to-reduce-generalization-error-in-deep-neural-networks-with-activity-regularization-in-keras/

https://stats.stackexchange.com/questions/431898/l2-lambdas-in-keras-regularizers

**Network configuration selected from the above is as below**

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [15, 1]                                 # try different depth and width
activationFuncEachLayer = ['tanh', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

print("'Mean F1 score : % 10.5f" %(evaluateTheModel()))

**L1 regularizer with different lambda values**

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [15,1]                                 # try different depth and width
activationFuncEachLayer = ['tanh', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l1(0)                       # try l1 and l2 with different lambda

for i in range(-5,5):
    regularizerFunc = tf.keras.regularizers.l1(10**i)
    print("'Regularizor : l1 with lambda : % 10.5f , Mean F1 score : % 10.5f" %(10**i, evaluateTheModel()))

**L2 regularizer with different lambda values**

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [15,1]                                 # try different depth and width
activationFuncEachLayer = ['tanh', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

for i in range(-5,5):
    regularizerFunc = tf.keras.regularizers.l2(10**i)
    print("'Regularizor : l2 with lambda : % 10.5f , Mean F1 score : % 10.5f" %(10**i, evaluateTheModel()))